# 書き込み誤差検討

In [ ]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
%env TOKEN=*************************************
! git clone https://$$TOKEN@github.com/konnitiha3/MOD2NN.git

import sys
sys.path.append('/content/MOD2NN')

from google.colab import drive
drive.mount('/content/drive')

In [36]:
import sys
import re
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import stats

from Faraday.two_dim.module.lib.layers import *
from Faraday.two_dim.module.lib import regularizer

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.0
Python: 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]


In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]

In [ ]:
model_path = ""
model = tf.keras.models.load_model(model_path)
eval = model.evaluate(x_test, y_test)

pattern = r'mo'

mo_layers = []
save = False
each_save = False
for layer in model.layers:
  result = re.match(pattern, layer.name)
  if result:
      mo_layers.append(layer)

seeds = np.arange(1, 11)
for seed in seeds:
    for layer in mo_layers:
        np.random.seed(seed)
        bi_dist = stats.bernoulli.rvs(p=0.9, size=(100,100)) * 2 - 1
        w = layer.get_weights()
        bi_w = np.where(w[0]>0, np.pi/2, -np.pi/2)
        w[0] = bi_w * bi_dist
        layer.set_weights(w)

    bi_eval = model.evaluate(x_test, y_test)